## Application Package publication on Resource Manager

Register a CWL application package in a resource manager catalog

In [ ]:
import sys
import yaml
import os
from utils.helpers import Atom
from lxml import etree
import datetime
import hashlib
import getpass
import requests
from requests.auth import HTTPBasicAuth
%load_ext autoreload
%autoreload 2

### Common Workflow Language (CWL) application package
Loaded from file

In [ ]:
# CWL file defining the application
# ---------------------------------
application_package = '../data/s-expression-cwl.cwl'
# ---------------------------------
with open(application_package, 'r') as stream:
    try:
        cwl = yaml.safe_load(stream)
        stream.seek(0)
        cwl_content = ''.join(stream.readlines())
    except yaml.YAMLError as exc:
        raise

Wrap the CWL definition in ATOM feed

In [ ]:
# unique id for application resource record
m = hashlib.md5()
m.update(cwl_content.encode('utf-8'))
identifier = m.hexdigest()
# Create atom representation
for elem in cwl['$graph']:
    if elem['class'] == 'Workflow':
        workflow = elem
template = '''<?xml version="1.0"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <entry>
    <title type="text"></title>
    <summary type="html"></summary>
    <date xmlns="http://purl.org/dc/elements/1.1/"></date>
    <identifier xmlns="http://purl.org/dc/elements/1.1/"></identifier>
  </entry>
</feed>'''
atom_template = Atom(etree.fromstring(template))
atom_template.set_identifier(identifier)
atom_template.set_title_text(workflow['label'])
atom_template.set_summary_text(workflow['doc'])
atom_template.set_dcdate(datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'))
offering = atom_template.create_offering(code='http://www.opengis.net/eoc/applicationContext/cwl',
                                         content=cwl_content)
atom_template.add_offering(offering)

### Publish the application as a record in the Resource Catalogue

In [ ]:
# Resource Catalogue endpoint
# ---------------------------
rm_endpoint = 'https://catalog.terradue.com/eoepca-services'
# ---------------------------
# Credentials etc.
username = 'rconway'
api_key = "AKCp8hyEYSMqk4swUndyCDVZW7g3XFYfvE2xVFQEdYagJLqVJY1qKbXcmStLoUFEyEhL5tKcY"
headers = {'Content-Type': 'application/atom+xml','Accept': 'application/json'}
auth = HTTPBasicAuth(username, api_key)

# Publish (POST) to the Resource Catalogue
# ----------------------------------------
r = requests.post(rm_endpoint, data=atom_template.to_string().decode('utf-8'), headers=headers, auth=auth)
# ----------------------------------------
print(r.status_code, r.reason)

Save it for future reference:

In [ ]:
with open('s-expression.xml', 'w') as f:
    f.write(b'<?xml version="1.0" encoding="UTF-8"?>\n'.decode('utf-8'))
    f.write(etree.tostring(atom_template.root, 
                           pretty_print=True).decode('utf-8'))